In [1]:
import matplotlib
%matplotlib tk
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%autosave 180

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import sys
sys.path.append('..')

# visualize results module
from visualize import visualize_svm as Visualize

import scipy
from tqdm import tqdm

Autosaving every 180 seconds


<ipython-input-1-9bc938cb63d7>:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
#############################################
#############################################
#############################################

class PCA_Analysis():

    def __init__(self):
        self.clr_ctr = 0
        
        
def get_pca_object_and_all_points(pa):

    fname_pickle = os.path.join(pa.root_dir, pa.animal_id,'tif_files',pa.session,
                                pa.session+"_pca_scatter_plot.pk")

    fname_all_points = os.path.join(pa.root_dir, pa.animal_id,'tif_files',pa.session,
                                pa.session+"_all_points.npy")
    X_30 = []
    for k in range(0,pa.X.shape[0]-100,pa.sliding_window):
        X_30.append(pa.X[k:k+pa.sliding_window])

    #
    X_30 = np.array(X_30)
    X_30 = X_30.reshape(X_30.shape[0],-1)
    print(" X data using : ", pa.sliding_window, " number of frames ", X_30.shape)

    if os.path.exists(fname_all_points)==False:

        # PCA ON ALL DATA
        pca = PCA(n_components=pa.n_pca)
        pca.fit(X_30)
        print (" done fit ")

        # do
        all_points = pca.transform(X_30)
        print ("all points denoised: ", all_points.shape)

        with open(fname_pickle, 'wb') as f:
            pickle.dump(pca, f)

        np.save(fname_all_points,
                all_points)
    else:

        with open(fname_pickle, "rb") as f:
            pca = pickle.load(f)
        all_points = np.load(fname_all_points)

    return pca, all_points



def get_convex_hull(pa):

    ''' COMPUTE CONVEX HULL
        - 
    
    '''
    
    # do very basic KNN triage
    n_dim = 3

    if True:

        from sklearn.neighbors import NearestNeighbors
        triage_value = 0.001
        knn_triage_threshold = 100*(1-triage_value)

        #if pca_wf.shape[0] > 1/triage_value:
        temp_points = pa.all_points[:,:n_dim]
        print ("temp points: ", temp_points.shape)
        idx_keep = knn_triage(knn_triage_threshold, temp_points)
        idx_keep = np.where(idx_keep==1)[0]

        print ("# points kept: ", idx_keep.shape,
               " of total: ", pa.all_points.shape[0])

        temp_points = temp_points[idx_keep]

    ######################################
    print ("computing convex hull # dim: ", n_dim, temp_points.shape)
    hull_all = ConvexHull(temp_points)

    #
    ratio_cumsum = []
    ratio_single = []
    ratio_random_single = []
    ratio_random_cumulative = []
    print ("p lever: ", pa.p_lever.shape)
    for k in trange(0, pa.p_lever.shape[0],1):

        # single frame convex hull; (frame???, n_times_points, n_dimensions)
        points = pa.p_lever[k:k+1,:,:n_dim].squeeze()#.reshape(-1,3)
        try:
            hull11 = ConvexHull(points)
        except:
            continue
        ratio_single.append(hull11.volume/hull_all.volume)

        # cumulative convex hull
        points = pa.p_lever[:k+1,:,:n_dim].reshape(-1,n_dim)
        #if k<10:
        #    print ("cumulative: ", points.shape)
        hull1 = ConvexHull(points)
        ratio_cumsum.append(hull1.volume/hull_all.volume)

        #
        #idx = np.random.randint(0,temp_points.shape[0],p_lever.shape[1])
        id_ = np.random.randint(0,temp_points.shape[0]-500,1)
        idx = np.arange(id_,id_+pa.p_lever.shape[1])
        points_random = temp_points[idx,:n_dim]
        hull3 = ConvexHull(points_random)
        ratio_random_single.append(hull3.volume/hull_all.volume)

        #
        ratio_random_cumulative.append([])
        for q in range(10):
            if True:
                idx = np.random.randint(0,temp_points.shape[0],
                                        pa.p_lever.shape[1]*(k+1))
            else:
                ns = np.random.randint(0,temp_points.shape[0]-500,k+1)
                idx = []
                for s in range(ns.shape[0]):
                    idx.append(np.arange(ns[s],ns[s]+pa.p_lever.shape[1]))
                idx=np.hstack(idx)
            points_random = temp_points[idx]
            #if q==0 and k<10:
            #    print (points_random.shape)
            hull4 = ConvexHull(points_random)
            ratio_random_cumulative[k].append(hull4.volume/hull_all.volume)

    pa.ratio_single = np.array(ratio_single)[::-1]
    pa.ratio_cumsum = np.array(ratio_cumsum)[::-1]
    pa.ratio_random_single = np.array(ratio_random_single)[::-1]
    pa.ratio_random_cumulative = np.array(ratio_random_cumulative)[::-1]

    print (pa.ratio_random_cumulative.shape)
    print (pa.ratio_random_single.shape)

    fname_out = os.path.join(pa.root_dir,
                             pa.animal_id,
                             'tif_files',
                             pa.session,
                             pa.session+"_convex_hull.npz")

    np.savez(fname_out,
             ratio_single = ratio_single,
             ratio_cumsum = ratio_cumsum,
             ratio_random_single = ratio_random_single,
             ratio_random_cumulative = ratio_random_cumulative
             )

    return pa


        
def knn_triage(th, pca_wf):

    tree = cKDTree(pca_wf)
    dist, ind = tree.query(pca_wf, k=6)
    dist = np.sum(dist, 1)

    idx_keep1 = dist <= np.percentile(dist, th)
    return idx_keep1


def triage_distributions(X_pca):

    # use knn triage to remove most outlier points
    triage_value = 0.01
    thresh = 100*(1-triage_value)

    # apply knn to all points
    #temp_points = X[:,:2]

    idx_keep = knn_triage(thresh, X_pca)
    idx_keep = np.where(idx_keep==1)[0]
    #all_points_knn = temp_points[idx_keep]
    
    X = X_pca[idx_keep]
    
    return X

In [3]:
#################################################################
#################################################################
#################################################################
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']

times = [600,900]


# 
fname = '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_12secLockout_[]bodyfeats.npz'
data = np.load(fname,allow_pickle=True)
trials = data['trials']
trials = np.vstack(trials)[:,times[0]:times[1]]
print (trials.shape)

# 


(310, 300, 16)


In [16]:
#############################################
#############################################
#############################################

# steps:
# 1. get multiple sessoin locaNMF traces 
# 2. chunk them into 30 frame bits
# 3. 

# 
import pickle
from scipy.spatial import cKDTree
from scipy.spatial import ConvexHull
from tqdm import trange, tqdm

root_dir = '/media/cat/4TBSSD/yuki/'

#
pa = PCA_Analysis()
pa.root_dir = root_dir #

#
pa.use_pca_data = True    # this uses the PCA denoised STMs not Raw data!
pa.recompute = True
pa.n_pca = 20
pa.sliding_window = 30    # how many frames to take into analysis window
pa.n_frames = 300          # how many frames back in time to analyze:


animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']

plt.figure(figsize=(10,10))

ctr2=0
for animal_id in animal_ids:

    ax = plt.subplot(3,2,ctr2+1)
    pa.animal_id = animal_id

    if animal_id != 'AQ2':
        pa.session = animal_id+'pm_Mar30_30Hz'
    else:
        pa.session = animal_id + 'am_Mar30_Week4_30Hz'

    ###########################################################
    ##### Load wholestack of locanmf data and run PCA #########
    ###########################################################
    # load wholstacks of data; 
    # may wish to load multiple sessions
    fname_data = os.path.join(pa.root_dir,
                              pa.animal_id,
                              'tif_files',
                              pa.session,
                              pa.session+'_locanmf_wholestack.npz')

    pa.X = np.load(fname_data,allow_pickle=True)['whole_stack'].T[500:-500]  # remove the beginning and end due to imaging issues
    print (pa.X.shape)

    # run PCA
    X_30 = []
    for k in range(0,pa.X.shape[0]-100,pa.sliding_window):
        X_30.append(pa.X[k:k+pa.sliding_window])

    #
    X_30 = np.array(X_30)
    X_30 = X_30.reshape(X_30.shape[0],-1)
    print(" X data using : ", pa.sliding_window, " number of frames ", X_30.shape)

    fname_pickle = fname_data[:-4]+"_pca_scatter_plot.pk"
    fname_all_points = fname_data[:-4]+"_all_points.npy"

    if os.path.exists(fname_all_points)==False:
        from sklearn.decomposition import PCA

        # PCA ON ALL DATA
        pca = PCA(n_components=pa.n_pca)
        pca.fit(X_30)
        print (" done fit ")

        # do
        all_points = pca.transform(X_30)
        print ("all points denoised: ", all_points.shape)

        with open(fname_pickle, 'wb') as f:
            pickle.dump(pca, f)

        np.save(fname_all_points,
                all_points)
    else:

        with open(fname_pickle, "rb") as f:
            pca = pickle.load(f)
        all_points = np.load(fname_all_points)

    ####################################################
    ########## Load locked out super sessions ##########
    ####################################################
    all_points = triage_distributions(all_points)
    n_dim = 3
    print ("computing convex hull # dim: ", n_dim, all_points[:,:n_dim].shape)
    hull_all = ConvexHull(all_points[:,:n_dim])
    print ("all points hull volume: ",hull_all.volume)

    # load the lockout data
    fname_data = os.path.join(pa.root_dir,
                              pa.animal_id,
                              'super_sessions',
                              'alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats.npz')

    data = np.load(fname_data,allow_pickle=True)
    trials = np.vstack(data['trials'])
    print ("# sessions: ", trials.shape," example session: ", trials[0].shape)
    start = 600
    end = 900
    trials = trials[:,start:end]
    print ("trials: -10sec to 0sec (# of trials, # of frames, # of ROIs): ", trials.shape)

    ####################################################
    ### apply previous computed PCA on superssession ###
    ####################################################
    aucs_norm = []
    session_n = []
    ctr = 0
    n_trials = 100
    for e in trange(0,trials.shape[0],25):
        ratio_cumsum = []
        for k in range(0,300,10):
            temp = trials[e:e+n_trials,k:k+pa.sliding_window]
            if temp.shape[0]!=n_trials or temp.shape[1]!=pa.sliding_window:
                break

            #print ("temp: ", temp.shape)
            temp = temp.reshape(n_trials,-1)
            # print ("temp reshaped: ", temp.shape)

            pca_lever = pca.transform(temp)
            #print ("pca_lever: ", pca_lever.shape)

            # cumulative convex hull
            points = pca_lever[:,:n_dim].reshape(-1,n_dim)
            hull1 = ConvexHull(points)
            ratio_cumsum.append(hull1.volume/hull_all.volume)
            #print ("hull1 volume: ", hull1.volume)

        try:
            #print ("ratio: ", ratio_cumsum)
            #print ("max: ", np.max(ratio_cumsum))
            aucs_norm.append((ratio_cumsum/np.max(ratio_cumsum)).sum())
            session_n.append(ctr)
        except:
            pass

        ctr+=1
        #plt.scatter(np.arange(len(ratio_cumsum)), ratio_cumsum)

    ######################################################
    ######################################################
    ######################################################

    plt.scatter(session_n, 
                aucs_norm, 
                s=300,
                edgecolor='black',
                #label=animal_id,
                c='red',
                alpha=.65)


    from sklearn.linear_model import LinearRegression
    from scipy import stats

    model = LinearRegression()
    y=np.array(aucs_norm).reshape(-1, 1)
    x = np.array(session_n).reshape(-1, 1)
    model.fit(x, y)

    x2 = np.arange(0,y.shape[0],1).reshape(-1, 1)
    y_pred = model.intercept_ + model.coef_ * x2

    # compute correlation between time and location
    corr = stats.pearsonr(x.squeeze(),y.squeeze())
    print ("pearson correlation: ", corr)

    plt.plot(x2, y_pred, 
             label= "pcorr: "+str(round(corr[0],5))+"\npval: "+
                             str(round(corr[1],5)),
             c='black',
             linewidth=10)


    plt.legend()
    #plt.ylim(bottom=0)
    plt.xlim(x[0],x[-1])
    plt.xticks([])
    plt.yticks([])
    
    #
    ctr2+=1
    
# 
if True:
    plt.savefig('/home/cat/AUCs_all_animals.svg')
    plt.close()
else:
    plt.show()
        

(39000, 16)
 X data using :  30  number of frames  (1297, 480)
computing convex hull # dim:  3 (1284, 3)
all points hull volume:  28638367411.027306


 10%|▉         | 2/21 [00:00<00:01, 17.39it/s]

# sessions:  (503, 1800, 16)  example session:  (1800, 16)
trials: -10sec to 0sec (# of trials, # of frames, # of ROIs):  (503, 300, 16)


100%|██████████| 21/21 [00:00<00:00, 21.74it/s]


pearson correlation:  (0.8009287445429161, 0.00011181261176070025)
(39000, 16)
 X data using :  30  number of frames  (1297, 480)
computing convex hull # dim:  3 (1284, 3)
all points hull volume:  16323950213.598078


...

pearson correlation:  (-1.0, 1.0)
(39000, 16)
 X data using :  30  number of frames  (1297, 480)
computing convex hull # dim:  3 (1284, 3)
all points hull volume:  4215382219.1976748


  8%|▊         | 2/24 [00:00<00:01, 18.29it/s]

# sessions:  (594, 1800, 16)  example session:  (1800, 16)
trials: -10sec to 0sec (# of trials, # of frames, # of ROIs):  (594, 300, 16)


100%|██████████| 24/24 [00:01<00:00, 23.79it/s]


pearson correlation:  (0.6834805976249893, 0.000892984335025864)


In [42]:
x2 = triage_distributions(all_points)
plt.figure()
plt.scatter(x2[:,0],
            x2[:,1])
plt.show()

In [19]:
###############################################
###############################################
###############################################
locanmf = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Mar30_30Hz/IA1pm_Mar30_30Hz_locanmf_wholestack.npz',
                  allow_pickle=True)

whole_stack = locanmf['whole_stack'].T
print (whole_stack.shape)

(40000, 16)


In [273]:
###########################################
########### HELPER FUNCTIONS ##############
###########################################
def load_reward_times(root_dir,
                      animal_id,
                      sessions,
                      width):
    # load rewarded and nonrewarded times in miliseconds
    rtimes = np.loadtxt(os.path.join(root_dir,
                                  animal_id,
                                  'tif_files',
                                  sessions[0],
                                  'rewarded_times.txt'))*1E3

    #
    nonrtimes = np.loadtxt(os.path.join(root_dir,
                                  animal_id,
                                  'tif_files',
                                  sessions[0],
                                  'nonrewarded_times.txt'))*1E3

    # add all data togheter
    # rtimes = np.concatenate((rtimes, nonrtimes))

    # 
    rtimes_random = []
    for k in range(rtimes.shape[0]):
        temp = rtimes[k] 
        rr = (np.random.rand(1)*50-25)*1E3
        #print (temp, rr)
        rtimes_random.append(temp+rr)

    # convolve all data with gaussian

    start = -10*1E3
    end = 10*1E3
    x = np.arange(start, end, 1) # entire range of x, both in and out of spec
    std = 1.5*1E3  # std in miliseconds
    mu = 0
    y = scipy.stats.norm.pdf(x,mu,std) 

    t = np.zeros(int(1400*1E3))
    for r in tqdm(rtimes):
        try:
            t[int(r+start):int(r+end)]+=y
        except:
            pass

    t_random = np.zeros(int(1400*1E3))
    for r in tqdm(rtimes_random):
        try:
            t_random[int(r+start):int(r+end)]+=y
        except:
            pass

    #####################################
    # stack segments on top of each other
    stack = []
    stack_random = []
    for r in tqdm(rtimes):
        temp = t[int(r-width):int(r+width)]
        if temp.shape[0]==(2*width):
            stack.append(temp)

        temp = t_random[int(r-width):int(r+width)]
        if temp.shape[0]==(2*width):
            stack_random.append(temp)


    stack = np.vstack(stack).T
    stack_random = np.vstack(stack_random).T
    print ("stacK: ", stack.shape)

    ######################################
    # sample back down to 30Hz
    t = t[::30]
    x = np.arange(t.shape[0])/30.
    
    return x, t, stack, stack_random

#
def plot_n_trials_per_session(root_dir,
                             animal_id,
                             session_id):

    arr = []
    ctr=0
    for animal_id in animal_ids:
        sessions = Visualize.get_sessions(root_dir,
                                          animal_id,
                                          session_id)


        # load rewarded and nonrewarded times in miliseconds
        rtimes = np.loadtxt(os.path.join(root_dir,
                                      animal_id,
                                      'tif_files',
                                      sessions[0],
                                      'rewarded_times.txt'))

        # load rewarded and nonrewarded times in miliseconds
        n_trials = []
        for session in tqdm(sessions):
            try:
                import warnings
                with warnings.catch_warnings():
                     warnings.simplefilter("ignore")
                     #data = np.loadtxt(myfile, unpack=True)
                     rtimes = np.loadtxt(os.path.join(root_dir,
                                          animal_id,
                                          'tif_files',
                                          session,
                                          'rewarded_times.txt'), unpack=True)
                if rtimes.shape[0]>30:
                    n_trials.append(rtimes.shape[0])
            except:
                pass
        n_trials = np.array(n_trials)
        idx = np.where(np.isnan(n_trials)==False)[0]
        n_trials = n_trials[idx]

        arr.append(n_trials)
        ctr+=1
        #if ctr>1:
        #    break

    #####################################
    linestyles = ['-','-','-']
    pvals = [0.05,0.01,0.001,0.0001,0.00001]
    my_dict = dict(
                    M1 = arr[0], 
                    M2 = arr[1], 
                    M3 = arr[2], 
                    M4 = arr[3], 
                    M5 = arr[4], 
                    M6 = arr[5],                
                  )

    # 
    #cons.append(edts[1])
    data = pd.DataFrame.from_dict(my_dict, orient='index')
    data = data.transpose()

    ###############################################
    ########## PLOT SCATTER #######################
    ###############################################
    fig = plt.figure(figsize=(24,8))
    clrs_local = ["black","black","black","black","black","black",] #,"red"]
    for i,d in enumerate(data):
        y = data[d]
        #print ('y: ', y)
        x = np.random.normal(i+1, 0.04, len(y))
        if True:
            plt.plot(x, y, 
                 mfc =clrs_local[i], 
                 mec='k', 
                 ms=7, 
                 marker="o", 
                 linestyle="None",
                 alpha=.8
                )
        else:
            plt.scatter(x, y, 
                   c=clrs_local[i],
                   edgecolor='black',
                   s=200,
                   alpha=.7)


    ###############################################
    ########## PLOT  #######################
    ###############################################
    flierprops = dict(marker='o', 
                      #markerfacecolor='g', 
                      markersize=10000,
                      linestyle='none', 
                      markeredgecolor='r')

    #
    bplot = data.boxplot(showfliers=True,
                flierprops=flierprops,
                grid=False) 

    plt.ylabel("# of trials in session")

    #
    plt.ylim(29,200)  
    
#   
def plot_average_convolution(width, stack, stack_random):

    plt.figure(figsize=(8,8))
    t= np.arange(-width,width,1)/1E3
    plt.plot(t,stack,c='blue',alpha=.01)
    # plt.plot(t,stack[:,0],c='black',alpha=0, label='singletrial')
    signal = stack.mean(1)
    plt.plot(t,signal,
             c='darkblue',
             linewidth=3,
             label="Session average")
    #
    if True:
        signal_random = stack_random.mean(1)
        plt.plot(t,signal_random,
                 c='darkred',
                 linewidth=3,
                 alpha=.7,
                 label="Session average - random")


        
    plt.plot([-3,-3],
             [0,0.00030],'--',
            c='black')
    plt.plot([3,3],
             [0,0.00030],'--',
            c='black')
    plt.xlim(t[0],t[-1])
    xticks=[-50,-40,-30,-20,-10,-3,0,
            3,10,20,30,40,50]
    plt.xticks(xticks)
    plt.xlabel("Time (sec)")
    plt.yticks([])

    plt.title(sessions[0]+ " # of trials: "+ str(stack.shape[1]))
    
    
    
    
#    
# def plot_average_convolution_hilbert(width, stack, stack_random):

#     # this function works on the simulated neural signals; so thereare no ROIs
    
#     plt.figure(figsize=(8,8))
#     print ("stack: ", stack.shape)
#     mean = stack.mean(1)
#     print ("means: ", mean.shape)
    
#     #
#     analytic_signal = hilbert(mean)
#     amplitude_envelope = np.abs(analytic_signal)
#     #amplitude_envelope = butter_lowpass_filter(amplitude_envelope,.5,30)

#     start = int(0*1E3)
#     end = int(1050*1E3)
    
#     # 
#     t = mean[start:end]
#     x = np.arange(t.shape[0])/1E3
#     amplitude_envelope = amplitude_envelope[start:end]

#     #
#     plt.plot(x, t, c='darkblue', linewidth=5, label='Average session neural signal')
#     plt.plot(x, amplitude_envelope, '--', c='black', linewidth=1, label="Hilbert transform envelope")


#     #
#     # plt.plot([x[0], x[-1]],[0,0],'--',c='black', linewidth=3)
#     # plt.plot([x[600], x[600]],[-5,7],'--',c='black', linewidth=3)

    
#     #plt.plot(tt,means[:,feature_id],c='blue')
#     #plt.plot(t,random.mean(0)[:,4],c='red')
#     #plt.xlim(-30,3)
#     plt.title(animal_id+" " +str(lockout)+"sec")
#     #ctr+=1
    
#     #xticks=[-50,-40,-30,-20,-10,-3,0,
#     #        3,10,20,30,40,50]
#     #plt.xticks(xticks)
#     #plt.xlim(t[0],t[-1])
    
    
# #     plt.plot([-3,-3],
# #              [0,0.00030],'--',
# #             c='black')
# #     plt.plot([3,3],
# #              [0,0.00030],'--',
# #             c='black')
   
#     plt.xlabel("Time (sec)")
#     plt.yticks([])

#     plt.title(sessions[0]+ " # of trials: "+ str(stack.shape[1]))

In [293]:
###########################################
############ LOAD LEVER PULL TIMES ########
###########################################
root_dir = '/media/cat/4TBSSD/yuki/'
animal_id = 'IA1'
session_id = 'IA1pm_Feb2_30Hz'
session_id = 'IA1pm_Feb16_30Hz'
session_id = 'IA1pm_Feb3_30Hz'
session_id = 'IA1pm_Feb9_30Hz'

animal_id = 'IA2'
# session_id = 'IA2pm_Mar14_30Hz'
# session_id = 'IA2pm_Feb29_30Hz'
session_id = 'IA2pm_Feb3_30Hz'

animal_id = 'IA3'
# session_id = 'IA3pm_Apr1_30Hz'
session_id = 'IA3pm_Feb4_30Hz'


#animal_id = 'IJ1'
# # session_id = 'IJ1am_Mar10_30Hz'
# #session_id = 'IJ1pm_Mar2_30Hz'
# session_id = 'IJ1pm_Mar3_30Hz'
#session_id = 'IJ1pm_Feb11_30Hz'

# animal_id = 'IJ2'
# session_id = 'IJ2pm_Apr5_30Hz'
# session_id = 'IJ2pm_Feb9_30Hz'

#animal_id = 'AQ2'
# session_id = 'AQ2am_Apr1_Week4_30Hz'
#session_id = 'AQ2am_Dec30_30Hz'
# session_id = 'AQ2am_Apr7_Week5_30Hz'
# session_id = 'AQ2am_Apr1_Week4_30Hz'

# session_id = 'all'
# session = session_id  # or 'all'
sessions = Visualize.get_sessions(root_dir,
                                  animal_id,
                                  session_id)

# sessions = [sessions[2]]
print ("# sessions: ", sessions)


#
width= 50*1E3  # in milliseconds

x, t, stack, stack_random = load_reward_times(root_dir,
                                               animal_id,
                                               sessions,
                                               width)



100%|██████████| 56/56 [00:00<00:00, 90968.64it/s]

# sessions:  ['IA3pm_Feb4_30Hz']
stacK:  (100000, 54)


In [168]:
#####################################################################
####### PLOT CONVOLUTION (LEVER PULL, GAUSSIAN) ACROSS SESSION ######
#####################################################################
plt.figure(figsize=(24,8))
plt.plot(x,t)
plt.xlabel("Time (sec)")
plt.ylabel("Convolution (lever pull time, gaussian)")
plt.yticks([])
plt.xlim(x[0],x[-1])
if False:
    plt.savefig('/home/cat/rtimesgaussian.svg')
    plt.close()
else:
    plt.show()

In [297]:
#######################################
####### PLOT AVERAGE CONVOLUTION ######
####################################### 
#     
plot_average_convolution(width,
                         stack,
                         stack_random)
plt.xlim(-20,5)
if True:
    plt.savefig('/home/cat/stacks.svg')
    plt.close()
else:
    plt.show()



In [225]:
################################################
####### PLOT AVERAGE NUMBER OF LEVER PULLS #####
################################################
# plot # of lever pulls per session per animal as scatter plot
import pandas as pd
root_dir = '/media/cat/4TBSSD/yuki/'

animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
session_id = 'all'

plot_n_trials_per_session(root_dir,
                         animal_id,
                         session_id)

#
if False:
    plt.savefig('/home/cat/n_trials_per_session.svg')
    plt.close()
else:
    plt.show()

100%|██████████| 44/44 [00:00<00:00, 1880.82it/s]
<ipython-input-188-a8a715d4c4bf>:95: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec9_30Hz/rewarded_times.txt"
  rtimes = np.loadtxt(os.path.join(root_dir,
100%|██████████| 110/110 [00:00<00:00, 1692.08it/s]


In [153]:
########################################################
#### PLOT READINESS POTENTIAL FOR LOCKED OUT DATA ######
########################################################
from scipy.signal import butter, lfilter, filtfilt, hilbert, chirp

def butter_lowpass(cutoff, nyq_freq, order=4):
    normal_cutoff = float(cutoff) / nyq_freq
    b, a = butter(order, normal_cutoff, btype='lowpass')
    return b, a

def butter_lowpass_filter(data, cutoff_freq, nyq_freq, order=4):
    # Source: https://github.com/guillaume-chevalier/filtering-stft-and-laplace-transform
    b, a = butter_lowpass(cutoff_freq, nyq_freq, order=order)
    y = filtfilt(b, a, data)
    return y



animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
animal_ids = ['IJ2']

clrs = ['darkblue','red']
lockouts = [0,21]  #

max_trials_in_animal = [399,   # 21 sec min times
                        385,
                        96,
                        137,
                        82,
                        318]


# max_trials_in_animal = [601,   # 12 sec min trials
#                         728,
#                         310,
#                         371,
#                         197,
#                         910]

#
plt.figure(figsize=(20,4))
ctr=0

# 
for animal_id in animal_ids:
    ax=plt.subplot(1,6,ctr+1)
    
    for ctr_color, lockout in enumerate(lockouts): 

        data = np.load('/media/cat/4TBSSD/yuki/'+animal_id+'/super_sessions/alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats.npz',
                       allow_pickle=True)

        names = data['names']
        behaviors = data['behaviors']
        trials = data['trials']

        temp = []
        for tr in trials:
            if tr.shape[0]<=30:
                temp.append(tr)
        
            
            
        trials = np.vstack(temp)
        print ("trials: ", trials.shape)
        random = data['random']
        random = np.vstack(random)
        


        #
        if False:
            maxes = max_trials_in_animal[ctr]
            #n_tr = min(trials.shape[0],300)
            idx = np.random.choice(np.arange(trials.shape[0]),maxes, replace=False)
        else:
            idx = np.arange(trials.shape[0])
        
        means = trials[idx].mean(0)
        tt= np.arange(trials.shape[1])/30-30

        #
        feature_id = 4

        #
        x = tt.copy()

        #
        # analytic_signal = hilbert(means[:,feature_id])
        # amplitude_envelope = np.abs(analytic_signal)
        # amplitude_envelope = butter_lowpass_filter(amplitude_envelope,.5,30)

        start = 0
        end = 1050
        t = means[:,feature_id][start:end]
        
        #t = butter_lowpass_filter(t,1,30)
        
        t = t/np.max(t)

        # 
        x = x[start:end]
        #amplitude_envelope = amplitude_envelope[start:end]

        plt.plot(x, t, c=clrs[ctr_color], 
                 linewidth=5, 
                 alpha=0.6,
                 label='Average session neural signal')
        # plt.plot(x, amplitude_envelope, '--', c='black', linewidth=1, label="Hilbert transform envelope")


        #
        plt.plot([x[0], x[-1]],[0,0],'--',c='black', linewidth=3)
        plt.plot([x[900], x[900]],[-1,1],'--',c='black', linewidth=3)


        #plt.plot(tt,means[:,feature_id],c='blue')
        #plt.plot(t,random.mean(0)[:,4],c='red')
        plt.xlim(-20,3)
        #plt.title(animal_id+" " +str(lockout)+"sec")
        plt.yticks([])
        plt.xticks([])
    ctr+=1
    
if True:
    plt.savefig('/home/cat/allaverages_'+str(lockouts)+'.svg')
    plt.close()
else:
    plt.show()


trials:  (165, 1800, 16)
trials:  (82, 1800, 16)


In [110]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_0secLockout_[]bodyfeats.npz',
               allow_pickle=True)

behaviors = data['all_area_names']
for ctr,b in enumerate(behaviors[0]):
    print (ctr, b)


0 Retrosplenial area, dorsal part, layer 1
1 Retrosplenial area, lateral agranular part, layer 1
2 Primary visual area, layer 1
3 Primary somatosensory area, upper limb, layer 1
4 Primary somatosensory area, lower limb, layer 1
5 Primary somatosensory area, barrel field, layer 1
6 Secondary motor area, layer 1
7 Primary motor area, Layer 1
8 Primary motor area, Layer 1
9 Secondary motor area, layer 1
10 Primary somatosensory area, barrel field, layer 1
11 Primary somatosensory area, lower limb, layer 1
12 Primary somatosensory area, upper limb, layer 1
13 Primary visual area, layer 1
14 Retrosplenial area, lateral agranular part, layer 1
15 Retrosplenial area, dorsal part, layer 1


In [281]:
if True:
    d1 = np.load('/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_0secLockout_[]bodyfeats.npz',
                 allow_pickle=True)
    trials = d1['trials']
    print (trials.shape)

    sessions2 = d1['all_session_names']
    #print (sessions2)
    for k in range(len(sessions2)):
        if "Feb" in sessions2[k]:
            print (k,sessions2[k], trials[k].shape)    
 

(41,)
10 IA3pm_Feb1_30Hz (30, 1800, 16)
11 IA3pm_Feb2_30Hz (65, 1800, 16)
12 IA3pm_Feb3_30Hz (94, 1800, 16)
13 IA3pm_Feb4_30Hz (54, 1800, 16)
14 IA3pm_Feb9_30Hz (11, 1800, 16)
15 IA3pm_Feb11_30Hz (13, 1800, 16)
16 IA3pm_Feb12_30Hz (83, 1800, 16)
17 IA3pm_Feb15_30Hz (48, 1800, 16)
18 IA3pm_Feb16_30Hz (66, 1800, 16)
19 IA3pm_Feb17_30Hz (50, 1800, 16)
20 IA3pm_Feb18_30Hz (38, 1800, 16)
21 IA3pm_Feb19_30Hz (30, 1800, 16)
22 IA3pm_Feb22_30Hz (24, 1800, 16)
23 IA3pm_Feb23_30Hz (11, 1800, 16)
24 IA3pm_Feb26_30Hz (67, 1800, 16)
25 IA3pm_Feb29_30Hz (49, 1800, 16)


In [298]:
#############################################
#############################################
#############################################
session_id = 13
feature_id = 4
    
#
print (trials[session_id].shape)
means = trials[session_id].mean(0)

plt.figure(figsize=(8,8))
plt.plot([-3,-3],[-30,100],'--',c='black')
plt.plot([3,3],[-30,100],'--',c='black')
t=np.arange(means.shape[0])/30-30
plt.plot(t,means[:,feature_id])
plt.title(sessions2[session_id])
plt.xlim(-20,5)

if True:
    plt.savefig('/home/cat/example_oscillations.svg')
    plt.close()
else:
    plt.show()


(54, 1800, 16)
